In [ ]:
from matplotlib import pyplot
import pathlib
import pandas

In [ ]:
OUTPUT_DIR = pathlib.Path("../../output/").resolve()

# Data curation

## Distinct values

Are `<Appointment_ID, Patient_ID>` tuples unique?
If they aren't, then each row might not represent an appointment.

* In [`2e4a681`][1] we checked if `<Appointment_ID>` tuples were unique. They weren't.
* In [`bca8390`][2] we checked if `<Appointment_ID, Organisation_ID>` tuples were unique. They weren't.

[1]: https://github.com/opensafely/appointments-short-data-report/commit/2e4a681cc7aa7b4a90455bf82e42e8777ae40eee
[2]: https://github.com/opensafely/appointments-short-data-report/commit/bca8390250ef3ce6013847d23e38a5611391d65e

In [ ]:
pandas.read_csv(OUTPUT_DIR / "distinct_values" / "results.csv")

## Date range

What is the date range of each of the datetime columns in the `Appointment` table?
Do the maximum or minimum values suggest [default values][1] (`1900-01-01`) or proxy null values?

[1]: https://learn.microsoft.com/en-us/sql/t-sql/data-types/datetime-transact-sql?view=sql-server-2017

In [ ]:
pandas.read_csv(
    OUTPUT_DIR / "date_range" / "results.csv",
    index_col="column_name",
    parse_dates=["min_date", "max_date"],
)

## Number of rows by month

In [ ]:
# long, missing dates are not represented
num_rows_by_month = pandas.read_csv(
    OUTPUT_DIR / "num_rows_by_month" / "results.csv",
    index_col=["date", "column_name"],
    parse_dates=["date"],
)

In [ ]:
# wide, missing dates at the extremes of the range are represented as NaN
num_rows_by_month = num_rows_by_month.unstack().loc[:, "num_rows"]

In [ ]:
full_date_range = pandas.date_range(
    num_rows_by_month.index.min(),
    num_rows_by_month.index.max(),
    freq="MS",
    name=num_rows_by_month.index.name,
)

In [ ]:
# wide, missing dates within, and at the extremes of, the range are represented as NaN
num_rows_by_month = num_rows_by_month.reindex(full_date_range)

In [ ]:
# all missing dates are represented as NaN, which are represented as gaps in the line
num_rows_by_month.plot(subplots=True, figsize=(12, 12))

Show the earliest 12 dates for each `column_name`.

In [ ]:
# stack drops NaN
num_rows_by_month.stack().rename("num_rows").swaplevel().sort_index().groupby(
    "column_name"
).head(12).to_frame()

Show the latest 12 dates for each `column_name`.

In [ ]:
# stack drops NaN
num_rows_by_month.stack().rename("num_rows").swaplevel().sort_index().groupby(
    "column_name"
).tail(12).to_frame()

## Lead time

We define *lead time* as `StartDate - BookedDate`, in days.
We expect `BookedDate <= StartDate`.
We do not expect negative lead times.

In [ ]:
lead_time = pandas.read_csv(OUTPUT_DIR / "lead_time" / "results.csv")

In [ ]:
pyplot.figure(figsize=(18, 6))
pyplot.xlabel(lead_time["lead_time_in_days"].name)
pyplot.ylabel(lead_time["frequency"].name)
pyplot.bar(lead_time["lead_time_in_days"], lead_time["frequency"])